<a href="https://colab.research.google.com/github/Yiming-xie-2021/aiperson_utility/blob/main/colab_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Link Google Drive and download model to drive, so can fast load from drive.

In [2]:
# link drive
from google.colab import drive
drive.mount('/content/drive')

# download a specific model to the drive
if False:
  !git lfs install
  !git clone https://huggingface.co/unsloth/Qwen3-1.7B-unsloth-bnb-4bit /content/drive/MyDrive/models/Qwen3-1.7B-unsloth-bnb-4bit

# list of models under the path
import os

path = "/content/drive/MyDrive/models/"
for f in os.listdir(path):
    print(f)


Mounted at /content/drive
Qwen3-1.7B-unsloth-bnb-4bit


### Installation
make sure GPU is on

In [3]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install -q langchain langchain-community

In [4]:
# from unsloth import FastLanguageModel
# import torch

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Qwen3-1.7B-unsloth-bnb-4bit",
#     max_seq_length = 2048,
#     load_in_4bit = True,
#     load_in_8bit = False,
#     full_finetuning = False,
# )

from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/models/Qwen3-1.7B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
if False:
  messages = [
    {"role" : "user", "content" : "江泽民是谁，习近平又是谁，谢一鸣又是谁，他们之间的关系是什么？."}
  ]
  text = tokenizer.apply_chat_template(
      messages,
      tokenize = False,
      add_generation_prompt = True,
      enable_thinking = False
  )

  from transformers import TextStreamer
  _ = model.generate(
      **tokenizer(text, return_tensors = "pt").to("cuda"),
      max_new_tokens = 2000,
      temperature = 0.7, top_p = 0.8, top_k = 20,
      streamer = TextStreamer(tokenizer, skip_prompt = True),
  )

In [ ]:
# !pip install -q langchain langchain-community
# 导入 LangChain 的 LLM 基类
from langchain_core.language_models import LLM
from typing import Optional, List, Any

class MyHFLLM(LLM):
    # 保存模型和分词器
    model: Any
    tokenizer: Any
    enable_thinking: bool = False

    def _call(self, prompt: str, stop=None):
        # 把 LangChain传进来的 prompt 转换成 messages
        messages = [
            # {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=self.enable_thinking)

        inputs = self.tokenizer(text, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(**inputs, max_new_tokens=2000)
        gen_tokens = outputs[:, inputs["input_ids"].shape[-1]:]
        return self.tokenizer.decode(gen_tokens[0], skip_special_tokens=True).strip()


    @property
    def _identifying_params(self):
        # 用于 LangChain 内部标识
        return {"model": self.model.__class__.__name__}

    @property
    def _llm_type(self):
        # 自定义类型标识
        return "custom_hf"


# ---------------- 使用示例 ----------------

# 假设你已经加载好模型和分词器，例如：
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model = AutoModelForCausalLM.from_pretrained("你的模型路径").to("cuda")
# tokenizer = AutoTokenizer.from_pretrained("你的模型路径")

# 默认不启用思维模式
llm = MyHFLLM(model=model, tokenizer=tokenizer)

# 启用思维模式
llm_thinking = MyHFLLM(model=model, tokenizer=tokenizer, enable_thinking=True)

# 定义 LangChain 的 Prompt
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{question}")
])

# 把 Prompt 和自定义 LLM 组合成一个链
chain = prompt | llm

# 测试调用
print(chain.invoke({"question": "谁是江泽民？"}))


In [ ]:
chain_thinking = prompt | llm_thinking
print(chain_thinking.invoke({"question": ""}))